In [1]:
#topic modelling
import numpy as np
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import plotly as py
import plotly.graph_objs as go

init_notebook_mode(connected=True) #do not miss this line

from gensim import corpora, models, similarities
import pandas as pd
from pymongo import MongoClient
#load tweets data as corpus
corpus=[]
client = MongoClient('localhost', 27017)
db = client.Facebook
for doc in db.tweets.distinct('text'):
    corpus.append(doc)

F:\Users\pp\Anaconda2\lib\site-packages\gensim\utils.py:1167: UserWarning:

detected Windows; aliasing chunkize to chunkize_serial



In [2]:
import string
punc = string.punctuation

In [3]:
import gensim
import logging
import tempfile
import re
from nltk.tokenize import word_tokenize, TweetTokenizer
import os
TEMP_FOLDER = tempfile.gettempdir()
print('Folder "{}" will be used to save temporary dictionary and corpus.'.format(TEMP_FOLDER))

from gensim import corpora
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

from nltk.corpus import stopwords
stop_words_english = stopwords.words('english')
stop_words_english.extend(['amp', 'via', '2016', 'never', 'ever', 'still', 'every', 'due', 'null', 'hey', '2018', '2017', 'next'])
def clean_tweet(tweet):
    # Remove tickers
    tweet = re.sub(r'\$\w*','', tweet)
    # Remove URLs
    tweet = re.sub(r'https?:\/\/.*\/\w*','', tweet)
    #Remove RT
    tweet = re.sub(r'RT','', tweet)
    # Remove puncutation
    tweet = re.sub(r'[' + punc + ']+', ' ', tweet)
    # Tokenize text
    tokenizer = TweetTokenizer(strip_handles=True, reduce_len=True)
    tokens = tokenizer.tokenize(tweet)
    # Remove stopwords and single characters
    tokens = [i.lower() for i in tokens if not i.lower() in stop_words_english and len(i) > 1]
    return tokens

texts = []
for tweet in corpus:
    texts.append(clean_tweet(tweet))

dictionary = corpora.Dictionary(texts)
dictionary.save(os.path.join(TEMP_FOLDER, 'tw.dict'))  # store the dictionary, for future reference

Folder "c:\users\pp\appdata\local\temp" will be used to save temporary dictionary and corpus.


2018-04-16 17:33:36,127 : INFO : adding document #0 to Dictionary(0 unique tokens: [])
2018-04-16 17:33:36,634 : INFO : adding document #10000 to Dictionary(16232 unique tokens: [u'raining', u'youtoken', u'allama', u'unlawf', u'jenoleniczak']...)
2018-04-16 17:33:37,144 : INFO : adding document #20000 to Dictionary(24718 unique tokens: [u'europeancommission', u'andrewsullivan', u'jotases', u'woods', u'libertycopbuns']...)
2018-04-16 17:33:37,759 : INFO : adding document #30000 to Dictionary(32226 unique tokens: [u'europeancommission', u'andrewsullivan', u'pysops', u'jotases', u'woods']...)
2018-04-16 17:33:38,384 : INFO : adding document #40000 to Dictionary(38878 unique tokens: [u'europeancommission', u'andrewsullivan', u'pysops', u'jotases', u'woods']...)
2018-04-16 17:33:38,984 : INFO : built Dictionary(44440 unique tokens: [u'europeancommission', u'andrewsullivan', u'pysops', u'jotases', u'woods']...) from 49580 documents (total 463239 corpus positions)
2018-04-16 17:33:38,987 : IN

In [4]:
corpus = [dictionary.doc2bow(text) for text in texts]
corpora.MmCorpus.serialize(os.path.join(TEMP_FOLDER, 'tw.mm'), corpus)  # store to disk, for later use

from gensim import corpora, models, similarities
tfidf = models.TfidfModel(corpus) # step 1 -- initialize a model
corpus_tfidf = tfidf[corpus]  # step 2 -- use the model to transform vectors

total_topics = 10
lda = models.LdaModel(corpus, id2word=dictionary, num_topics=total_topics, chunksize=2000)
corpus_lda = lda[corpus_tfidf] # create a double wrapper over the original corpus: bow->tfidf->fold-in-lsi

2018-04-16 17:33:52,532 : INFO : storing corpus in Matrix Market format to c:\users\pp\appdata\local\temp\tw.mm
2018-04-16 17:33:52,538 : INFO : saving sparse matrix to c:\users\pp\appdata\local\temp\tw.mm
2018-04-16 17:33:52,542 : INFO : PROGRESS: saving document #0
2018-04-16 17:33:52,736 : INFO : PROGRESS: saving document #1000
2018-04-16 17:33:52,914 : INFO : PROGRESS: saving document #2000
2018-04-16 17:33:53,119 : INFO : PROGRESS: saving document #3000
2018-04-16 17:33:53,301 : INFO : PROGRESS: saving document #4000
2018-04-16 17:33:53,484 : INFO : PROGRESS: saving document #5000
2018-04-16 17:33:53,670 : INFO : PROGRESS: saving document #6000
2018-04-16 17:33:53,862 : INFO : PROGRESS: saving document #7000
2018-04-16 17:33:54,061 : INFO : PROGRESS: saving document #8000
2018-04-16 17:33:54,243 : INFO : PROGRESS: saving document #9000
2018-04-16 17:33:54,408 : INFO : PROGRESS: saving document #10000
2018-04-16 17:33:54,582 : INFO : PROGRESS: saving document #11000
2018-04-16 17:3

2018-04-16 17:34:43,490 : INFO : topic #4 (0.100): 0.051*"facebook" + 0.020*"data" + 0.010*"want" + 0.006*"cambridgeanalytica" + 0.006*"privacy" + 0.006*"fb" + 0.005*"know" + 0.005*"85" + 0.005*"youtube" + 0.005*"mining"
2018-04-16 17:34:43,496 : INFO : topic #9 (0.100): 0.081*"facebook" + 0.028*"zuckerberg" + 0.013*"mark" + 0.008*"hearing" + 0.008*"senate" + 0.007*"ceo" + 0.007*"like" + 0.006*"social" + 0.006*"youtube" + 0.006*"privacy"
2018-04-16 17:34:43,503 : INFO : topic diff=0.287910, rho=0.577350
2018-04-16 17:34:43,509 : INFO : PROGRESS: pass 0, at document #8000/49580
2018-04-16 17:34:50,588 : INFO : merging changes from 2000 documents into a model of 49580 documents
2018-04-16 17:34:51,473 : INFO : topic #1 (0.100): 0.075*"facebook" + 0.046*"zuckerberg" + 0.025*"mark" + 0.017*"us" + 0.016*"testimony" + 0.016*"day" + 0.013*"congress" + 0.013*"ceo" + 0.009*"markzuckerberg" + 0.007*"data"
2018-04-16 17:34:51,477 : INFO : topic #7 (0.100): 0.082*"facebook" + 0.014*"page" + 0.013*

2018-04-16 17:35:27,608 : INFO : topic #6 (0.100): 0.062*"facebook" + 0.019*"latest" + 0.015*"thanks" + 0.014*"zuckerberg" + 0.011*"daily" + 0.009*"hate" + 0.009*"social" + 0.009*"profiles" + 0.009*"media" + 0.009*"speech"
2018-04-16 17:35:27,611 : INFO : topic #7 (0.100): 0.083*"facebook" + 0.016*"get" + 0.013*"page" + 0.011*"instagram" + 0.010*"business" + 0.010*"gt" + 0.009*"socialmedia" + 0.009*"11" + 0.008*"update" + 0.007*"media"
2018-04-16 17:35:27,618 : INFO : topic diff=0.250026, rho=0.333333
2018-04-16 17:35:35,540 : INFO : -9.797 per-word bound, 889.5 perplexity estimate based on a held-out corpus of 2000 documents with 18062 words
2018-04-16 17:35:35,543 : INFO : PROGRESS: pass 0, at document #20000/49580
2018-04-16 17:35:42,095 : INFO : merging changes from 2000 documents into a model of 49580 documents
2018-04-16 17:35:43,233 : INFO : topic #6 (0.100): 0.061*"facebook" + 0.022*"latest" + 0.019*"thanks" + 0.013*"daily" + 0.013*"hate" + 0.013*"zuckerberg" + 0.010*"speech" +

2018-04-16 17:36:20,085 : INFO : topic #8 (0.100): 0.076*"facebook" + 0.027*"zuckerberg" + 0.011*"like" + 0.008*"people" + 0.007*"mark" + 0.007*"think" + 0.007*"good" + 0.005*"real" + 0.005*"would" + 0.005*"qanon"
2018-04-16 17:36:20,089 : INFO : topic #0 (0.100): 0.060*"facebook" + 0.023*"zuckerberg" + 0.014*"trump" + 0.010*"syria" + 0.010*"diamondandsilk" + 0.010*"maga" + 0.009*"censorship" + 0.007*"potus" + 0.007*"watch" + 0.006*"community"
2018-04-16 17:36:20,095 : INFO : topic diff=0.251650, rho=0.258199
2018-04-16 17:36:20,101 : INFO : PROGRESS: pass 0, at document #32000/49580
2018-04-16 17:36:25,326 : INFO : merging changes from 2000 documents into a model of 49580 documents
2018-04-16 17:36:26,331 : INFO : topic #2 (0.100): 0.043*"facebook" + 0.013*"check" + 0.012*"project" + 0.009*"top" + 0.008*"lifelog" + 0.008*"darpa" + 0.007*"smm" + 0.007*"campaign" + 0.006*"going" + 0.006*"website"
2018-04-16 17:36:26,334 : INFO : topic #7 (0.100): 0.093*"facebook" + 0.020*"instagram" + 0

2018-04-16 17:37:08,724 : INFO : topic #7 (0.100): 0.097*"facebook" + 0.021*"instagram" + 0.020*"social" + 0.019*"socialmedia" + 0.017*"twitter" + 0.016*"media" + 0.016*"marketing" + 0.013*"page" + 0.013*"business" + 0.011*"get"
2018-04-16 17:37:08,730 : INFO : topic #3 (0.100): 0.119*"facebook" + 0.037*"data" + 0.018*"news" + 0.014*"cambridge" + 0.014*"privacy" + 0.012*"analytica" + 0.010*"cambridgeanalytica" + 0.010*"zuckerberg" + 0.007*"scandal" + 0.006*"markzuckerberg"
2018-04-16 17:37:08,734 : INFO : topic diff=0.235978, rho=0.218218
2018-04-16 17:37:08,739 : INFO : PROGRESS: pass 0, at document #44000/49580
2018-04-16 17:37:13,155 : INFO : merging changes from 2000 documents into a model of 49580 documents
2018-04-16 17:37:14,084 : INFO : topic #3 (0.100): 0.121*"facebook" + 0.036*"data" + 0.018*"news" + 0.015*"privacy" + 0.014*"cambridge" + 0.011*"analytica" + 0.011*"cambridgeanalytica" + 0.009*"zuckerberg" + 0.008*"scandal" + 0.006*"read"
2018-04-16 17:37:14,086 : INFO : topic 

In [5]:
#Show first n important word in the topics:
lda.show_topics(total_topics,10)

[(0,
  u'0.055*"facebook" + 0.039*"ep" + 0.015*"zuckerberg" + 0.011*"diamondandsilk" + 0.011*"maga" + 0.011*"trump" + 0.010*"censorship" + 0.008*"stories" + 0.006*"watch" + 0.006*"link"'),
 (1,
  u'0.079*"facebook" + 0.053*"zuckerberg" + 0.037*"us" + 0.028*"comes" + 0.023*"mark" + 0.018*"follow" + 0.015*"congress" + 0.012*"markzuckerberg" + 0.010*"vote" + 0.008*"strat"'),
 (2,
  u'0.037*"facebook" + 0.014*"usa" + 0.011*"check" + 0.009*"campaign" + 0.007*"top" + 0.007*"project" + 0.007*"new" + 0.007*"podcast" + 0.007*"spy" + 0.006*"ways"'),
 (3,
  u'0.125*"facebook" + 0.031*"data" + 0.020*"ai" + 0.016*"privacy" + 0.016*"news" + 0.013*"cambridge" + 0.011*"analytica" + 0.010*"scandal" + 0.010*"cambridgeanalytica" + 0.008*"zuckerberg"'),
 (4,
  u'0.034*"facebook" + 0.010*"problem" + 0.009*"13" + 0.007*"block" + 0.007*"stay" + 0.006*"show" + 0.005*"tech" + 0.005*"story" + 0.005*"wethepeople" + 0.005*"updates"'),
 (5,
  u'0.103*"facebook" + 0.053*"data" + 0.024*"europe" + 0.024*"sign" + 0.02

In [ ]:
from collections import OrderedDict
data_lda = {i: OrderedDict(lda.show_topic(i,25)) for i in range(total_topics)}

In [ ]:
import pyLDAvis.gensim
pyLDAvis.enable_notebook()
panel = pyLDAvis.gensim.prepare(lda, corpus_lda, dictionary, mds='tsne')
panel